In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt


In [3]:

def get_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date, interval="1d")
    data['Date'] = data.index
    return data


In [4]:
data = pd.read_csv("nifty_200/ABB.csv")[:500]

In [5]:

def identify_inside_bars(df):
    df['InsideBar'] = (
        (df[['Close', 'Open']].max(axis=1) < df[['Close', 'Open']].shift(1).max(axis=1)) &
        (df[['Close', 'Open']].min(axis=1) > df[['Close', 'Open']].shift(1).min(axis=1))
    )
    return df

def calculate_emas(df, emas):
    for len in emas:
        df[f"EMA_{len}"] = df['Close'].ewm(span=len, adjust=False).mean()
    return df

def apply_strategy(df):
    df['Buy_Signal'] = np.where(df['InsideBar'] & (df['Close'] > df['High'].shift(1)), 1, 0)
    df['Sell_Signal'] = np.where(df['InsideBar'] & (df['Close'] < df['Low'].shift(1)), -1, 0)
    return df

def backtest_strategy(df, position_size):
    df['Position'] = df['Buy_Signal'] + df['Sell_Signal']
    df['Position'] = df['Position'].shift(1).fillna(0)
    
    df['Market_Return'] = np.log(df['Close'] / df['Close'].shift(1))
    df['Strategy_Return'] = df['Position'] * df['Market_Return']
    
    df['Trade_Return'] = df['Strategy_Return'] * position_size
    df['Cumulative_Trade_Return'] = df['Trade_Return'].cumsum()

    total_return = df['Cumulative_Trade_Return'].iloc[-1]
    num_trades = df['Position'].abs().sum()
    win_rate = (df['Trade_Return'] > 0).sum() / num_trades if num_trades > 0 else 0
    max_drawdown = df['Cumulative_Trade_Return'].max() - df['Cumulative_Trade_Return'].min()
    sharpe_ratio = df['Strategy_Return'].mean() / df['Strategy_Return'].std() * np.sqrt(252) if df['Strategy_Return'].std() > 0 else 0
    
    kpis = {
        'Total Return': total_return,
        'Number of Trades': num_trades,
        'Win Rate': win_rate,
        'Max Drawdown': max_drawdown,
        'Sharpe Ratio': sharpe_ratio
    }
    
    return kpis, df

def plot_results(df, ticker):
    plt.figure(figsize=(12, 8))
    plt.plot(df['Cumulative_Trade_Return'], label=f'{ticker} Strategy Return')
    plt.title(f'Inside Bar Trading Strategy Backtest for {ticker}')
    plt.legend()
    plt.show()

def backtest_multiple_stocks(tickers, start_date, end_date, position_size):
    all_kpis = []

    for ticker in tickers:
        data = get_data(ticker, start_date, end_date)
        data = identify_inside_bars(data)
        data = calculate_emas(data, emas) 
        data = apply_strategy(data)
        kpis, backtested_data = backtest_strategy(data, position_size)
        
        kpis['Ticker'] = ticker
        all_kpis.append(kpis)
        
        plot_results(backtested_data, ticker)
    
    kpi_df = pd.DataFrame(all_kpis)
    return kpi_df


In [6]:
df = identify_inside_bars(data)

In [8]:
df[df["InsideBar"]]

,Datetime,Open,High,Low,Close,Volume,InsideBar
15,2020-07-06 10:15:00,1010.55,1012.15,992.45,998.55,103625,True
19,2020-07-06 14:15:00,983.95,990.60,981.05,984.00,40907,True
27,2020-07-07 15:15:00,960.00,963.50,958.80,961.40,23864,True
50,2020-07-13 10:15:00,917.05,919.75,915.50,916.70,6494,True
62,2020-07-14 15:15:00,907.35,912.95,906.30,908.15,3525,True
93,2020-07-21 11:15:00,904.20,906.05,903.00,904.25,8482,True
102,2020-07-22 13:15:00,915.90,920.00,915.00,916.50,13425,True
134,2020-07-29 10:15:00,892.65,894.00,890.10,893.00,11772,True
141,2020-07-30 10:15:00,881.00,881.95,878.10,880.05,11656,True
142,2020-07-30 11:15:00,880.60,881.40,879.50,880.35,4925,True


In [15]:
data.iloc[0]

Datetime    2020-07-02 09:15:00
Open                      966.0
High                      975.0
Low                       955.1
Close                    963.25
Volume                    43559
Name: 0, dtype: object

In [18]:
inside_bars = []
consecutive_inside_bars = 0  # Initialize a counter for consecutive inside bars
historical_data = data
# Iterate through historical data to detect inside bars
for i in range(1, len(historical_data)):
    current_candle = historical_data.iloc[i]
    previous_candle = historical_data.iloc[i - 1]

    if (current_candle['High'] < previous_candle['High']) and (current_candle['Low'] > previous_candle['Low']):
        inside_bars.append(current_candle)
        consecutive_inside_bars += 1
    else:
        consecutive_inside_bars = 0  # Reset the counter if no inside bar

    # Check if there were three consecutive inside bars
    if consecutive_inside_bars == 3:
        print(f"Triple Inside Bar detected for on {current_candle['Datetime']}")
        consecutive_inside_bars = 0  # Reset the counter after detection


Triple Inside Bar detected for on 2021-01-06 12:15:00
Triple Inside Bar detected for on 2021-03-10 13:15:00
Triple Inside Bar detected for on 2021-04-16 15:15:00
Triple Inside Bar detected for on 2021-11-10 12:15:00
Triple Inside Bar detected for on 2023-06-06 13:15:00
Triple Inside Bar detected for on 2023-06-26 12:15:00
Triple Inside Bar detected for on 2023-07-21 12:15:00
Triple Inside Bar detected for on 2023-09-11 12:15:00
Triple Inside Bar detected for on 2023-11-07 12:15:00
Triple Inside Bar detected for on 2024-03-05 12:15:00


In [9]:
df[df['InsideBar']]

,Open,High,Low,Close,Adj Close,Volume,InsideBar
Datetime,,,,,,,
2022-08-10 14:15:00+05:30,514.799988,515.900024,513.700012,515.099976,515.099976,3508643,True
2022-08-11 12:15:00+05:30,520.299988,521.599976,520.150024,521.299988,521.299988,838146,True
2022-08-12 10:15:00+05:30,529.900024,530.250000,528.150024,529.750000,529.750000,1336419,True
2022-08-12 11:15:00+05:30,529.799988,530.400024,529.150024,529.799988,529.799988,1089727,True
2022-08-16 14:15:00+05:30,527.500000,528.000000,525.599976,526.400024,526.400024,1878918,True
...,...,...,...,...,...,...,...
2024-07-23 14:15:00+05:30,865.799988,871.750000,862.049988,863.450012,863.450012,3184395,True
2024-07-24 11:15:00+05:30,853.599976,855.750000,849.200012,854.849976,854.849976,2585772,True
2024-07-26 13:15:00+05:30,859.299988,862.700012,857.849976,859.500000,859.500000,1706221,True


In [ ]:

tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN']
position_size = 100000
start_date = '2020-01-01'
end_date = '2024-08-09'

kpi_df = backtest_multiple_stocks(tickers, start_date, end_date, position_size)
print(kpi_df)

print(kpi_df)

# Save KPIs to a file
kpi_df.to_csv('inside_bar_strategy_kpis.csv', index=False)
